# Description

The notebook automates the process of pushing a pair of scenarios (baseline and alternative) through the simulation and optimization pipeline, comparing outputs for each scenario. Key outputs include harvested area (hectares), harvested volume (m³), and growth (m³). The notebook also reports stand age class distribution, old growth attributes, and tree species diversity at the beginning and end of the planning horizon for each scenario. Additionally, it calculates biodiversity and socioeconomic indicators, while quantifying differences between baseline and alternative scenarios in terms of carbon stocks and net emissions over a customizable simulation horizon, set to 100 years by default (the system can easily be parametrized to simulate shorter or longer horizons).

In [1]:
import numpy as np
import math
import matplotlib.pyplot as plt
import pandas as pd
import ipywidgets as widgets
from IPython.display import display
import time
import pickle
import os
import geopandas as gpd
import ws3.forest, ws3.core
import csv
from functools import partial, wraps
import distance
import operator
import shutil
from IPython.display import display
import libcbm
from util import compile_events, cbm_report, compile_scenario_maxstock, plot_scenario_maxstock, run_cbm_emissionstock, run_scenario, plugin_c_curves, plugin_c_curves, cbm_report_both, compare_ws3_cbm, compare_ws3_cbm_both, track_system_stock, track_system_emission, compile_scenario_minemission, plot_scenario_minemission, kpi_age, kpi_species, cmp_c_ss, cmp_c_se, results_scenarios, bootstrap_ogi, compare_kpi_age, epsilon_computer, tradeoff_biodiversity_cs, tradeoff_hv_cs, tradeoff_hv_biodiversity, inventory_processing, curve_points_generator, fm_bootstrapper, carbon_curve_points

In [2]:
# Input paths
# shapefile_path = './data/shp_files/tsa17_test.shp/stands selection.shp'
shapefile_path = './data/shp_files/tsa45.shp/stands.shp'

In [3]:
stands_org = gpd.read_file(shapefile_path)
# stands_org = stands_org.head(800)
stands_org.head(100)

,theme0,thlb,au,ldspp,age2015,shape_area,geometry
0,tsa45,1,515,Amabilis fir,10,4.579024e-05,"POLYGON ((854570.138 1010944.616, 854592.478 1..."
1,tsa45,1,515,Amabilis fir,13,7.921784e-05,"POLYGON ((853897.648 1009113.376, 853897.488 1..."
2,tsa45,1,172,Amabilis fir,5,7.031440e-05,"POLYGON ((848195.858 1011350.746, 848224.398 1..."
3,tsa45,1,203,Western hemlock,26,4.812744e-06,"POLYGON ((836432.78 1014457.699, 836431.799 10..."
4,tsa45,1,185,Red alder,21,1.496789e-06,"POLYGON ((836451.88 1014445.901, 836450.407 10..."
...,...,...,...,...,...,...,...
95,tsa45,0,354,Paper birch,78,1.903457e-08,"POLYGON ((1566901.247 574451.159, 1566838.349 ..."
96,tsa45,0,354,Paper birch,88,2.507140e-08,"POLYGON ((1567959.804 575395.55, 1567665.373 5..."
97,tsa45,0,265,Subalpine fir,48,5.141971e-09,"POLYGON ((1594382.304 584782.668, 1594382.023 ..."
98,tsa45,0,312,Engelmann spruce,278,2.583513e-05,"POLYGON ((1602482.566 645703.62, 1602511.278 6..."


In [4]:
# Initialize the input parameters
base_year = 2020
horizon = 7
period_length = 10
max_age = 1000
n_steps = 100
tvy_name = 'totvol'
max_harvest = 1.0
case_study = 'ecotrust'
scenario_names = ['lowest carbon stock', 'business as usual', '40% of highest carbon stock', '60% of highest carbon stock', '20% of highest carbon stock', 'highest carbon stock']
obj_mode = 'max_hv'
hwp_pool_effect_value = 0
release_immediately_value = 0
displacement_effect = 0
clt_percentage = 0
credibility = 0
budget_input = 10000000

In [5]:
# Reading yld table and changing AU column type into integer
yld = pd.read_csv('./data/yld.csv')
yld['AU'] = yld['AU'].astype(int)


# Getting names and codes of canfi_species. This table will be used to add canfi_species column into yldmerged
canf = pd.read_csv('data/canfi_species_revised.csv')
canf = canf[['name','canfi_species']].set_index('name')

In [6]:
stands = inventory_processing(stands_org, canf)

In [7]:
curve_points_table = curve_points_generator(stands, yld, canf)

In [8]:
fm = fm_bootstrapper(base_year, horizon, period_length, max_age, stands, curve_points_table, tvy_name)

In [9]:
c_curves_p, c_curves_f = carbon_curve_points(fm)

/home/salar2/project/ecotrust-dss-/venv-3.10/lib/python3.10/site-packages/libcbm/resources/__init__.py:112: RuntimeWarning: untested linux distribution: Linux-6.11.0-19-generic-x86_64-with-glibc2.39
  warnings.warn(message, RuntimeWarning)
/home/salar2/project/ecotrust-dss-/venv-3.10/lib/python3.10/site-packages/libcbm/resources/__init__.py:112: RuntimeWarning: untested linux distribution: Linux-6.11.0-19-generic-x86_64-with-glibc2.39
  warnings.warn(message, RuntimeWarning)


In [10]:
# Plug carbon and old groth yield curves into forest model
plugin_c_curves(fm, c_curves_p, c_curves_f)
bootstrap_ogi(fm)

KeyboardInterrupt: 

In [ ]:
tradeoff_biodiversity_cs(fm, clt_percentage, hwp_pool_effect_value, displacement_effect, release_immediately_value, n=4, solver=ws3.opt.SOLVER_PULP)

In [ ]:
epsilon, cs_max = epsilon_computer(fm, clt_percentage, hwp_pool_effect_value, displacement_effect, release_immediately_value, n=4, solver=ws3.opt.SOLVER_PULP)
tradeoff_hv_cs(fm, clt_percentage, hwp_pool_effect_value, displacement_effect, release_immediately_value, epsilon, cs_max, n=4, solver=ws3.opt.SOLVER_PULP)

In [ ]:
tradeoff_hv_biodiversity(fm, clt_percentage, hwp_pool_effect_value, displacement_effect, release_immediately_value, n=4, solver=ws3.opt.SOLVER_PULP)